In [15]:
from typing import Dict, Optional, Any
import numpy as np

#import gymnasium as gym
#from gym import spaces
import torch
from gymnasium import spaces

import gym

from gymnasium.wrappers import TimeLimit
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv,SubprocVecEnv
import numpy as np


from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from imitation.policies.base import FeedForward32Policy, SAC1024Policy, NonTrainablePolicy, RandomPolicy, ZeroPolicy
from stable_baselines3.ppo import MlpPolicy
from imitation.policies.base import  NormalizeFeaturesExtractor



from stable_baselines3.common.evaluation import evaluate_policy



from imitation.rewards.reward_wrapper import RewardVecEnvWrapper
from imitation.rewards.reward_nets import BasicRewardNet, BasicShapedRewardNet

from imitation.util.networks import RunningNorm

class CustomEnv(gym.Env):
    def __init__(self, S, A, R, max_steps): #max_steps
        self.S = S #torch.tensor(S, dtype=torch.float32)  # Convert S to torch tensor
        self.A = A#torch.tensor(A, dtype=torch.float32)  # Convert A to torch tensor
        self.R = R  if R is not None else None #torch.tensor(R, dtype=torch.float32)  # Convert R to torch tensor if provided
        self.num_samples = self.S.shape[0]
        self.current_step = 0
        self.max_steps = max_steps
        # Action space: assume discrete actions for simplicity
        self.action_dim = self.A.shape[1]
        self.observation_dim = self.S.shape[1]
        self.action_space = spaces.box.Box(low=-1e6, high=1e6, shape=(self.action_dim,))
        
        # Observation space: separate S and Y with specific labels
        # obs_dim_S = self.S.shape[1]
        # observation_space = {}
        # for i in range(obs_dim_S):
        #     observation_space[f'S_{i+1}'] = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        
        self.observation_space = spaces.box.Box(low=-np.inf, high=np.inf, shape=(self.observation_dim,))
        


    def reset(self, seed: int = None, options: Optional[Dict[str, Any]] = None):
        super().reset(seed=seed, options=options)
        #self.state = self.observation_space.sample()
        #self.steps_taken = 0
        # self.current_step = 0  # Reset current step
        if self.max_steps == None:
            self.initiation_index = 0
        else:
            self.initiation_index = np.random.randint(0, self.S.shape[0] - self.max_steps)
        
        self.current_step = self.initiation_index
        self.state = self.S[self.current_step,:]
        return self.state, {}
    


    def step(self,action): # action
        #print(self.state)
        #print(action)
        action = torch.from_numpy(action.astype(np.float64) )
        
        #self.state = self.observation_space.sample()
        #print(action.shape)
        
        

        reward = self._compute_reward(action)# torch.mean(self.A[self.current_step,:] - action)**(-1)
        #reward = None
        #print(self.current_step)
        #self.steps_taken += 1
        self.state = self.S[self.current_step,:]
        #done = (self.current_step == self.obs_dim_S - 1)
        #print("done",done)
        # self.current_step = self.current_step + 1
        #self.current_step = (self.current_step + 1) % self.S.shape[0]
        #done = (self.current_step == self.num_samples - 1) #or (self.steps_taken >= self.max_steps)
        #
        #
        # print(self.current_step)
        #
        #
        self.current_step += 1
        if self.max_steps == None:
            done = (self.current_step == self.num_samples - 1)
        else:
            done = self.current_step >= self.max_steps + self.initiation_index
        # observation, reward, terminated, truncated, info
        return self.state, reward, done, False, {}
    def _compute_reward(self, action):
        #reward = torch.mean(self.A[self.current_step,:] - action)**(-1) # 先假設這樣寫，這邊要REWIEW
        return 0.0

In [19]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
n_samples = 5000
n_features = 53
n_informative = 52
n_targets = 27
noise = 0.1

X, y = make_regression(n_samples=n_samples,n_features=n_features,n_informative=n_informative,n_targets=n_targets, noise=noise,random_state=9)



# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=random_seed)
# X_val, _, y_val, _ = train_test_split(
#     X_train, y_train, test_size=0.33, random_state=random_seed)
# train_size = X_train.shape[0]
# val_size = X_val.shape[0]

In [18]:
# resample_iter = 5000
# S = []
# A = []
# for i in range(resample_iter):
#     random_seed= np.random.randint(0,1000)
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.33, random_state=random_seed)
#     A.append(X_train)
#     S.append(y_train)
# S = np.stack(S)
# A = np.stack(A)

In [20]:
S = y
A = X

In [21]:
S.shape

(5000, 27)

In [22]:
A.shape

(5000, 53)

In [23]:


# 假設 S 和 A 是你的狀態和動作矩陣
# SAMPLE一定邀依樣
# n_sample = 5000
# S = np.random.rand(n_sample, 9)
# A = np.random.rand(n_sample, 23)

S = torch.from_numpy(S)
A = torch.from_numpy(A)
#env = CustomEnv(S, A,R=None)

#env = TimeLimit(env, max_episode_steps=500)

# Create a vectorized environment for training with `imitation`


# Option A: use a helper function to create multiple environments
def _make_env(s=S,a=A,R=None,max_steps=None):
    
    """Helper function to create a single environment. Put any logic here, but make sure to return a RolloutInfoWrapper."""
    _env = CustomEnv(s,a,R,max_steps)
    #_env = TimeLimit(_env, max_episode_steps=max_steps)
    _env = RolloutInfoWrapper(_env)
    return _env

In [24]:
new_obs.shape

torch.Size([1, 27])

In [25]:
new_obs = np.random.randn(1,S.shape[1])
new_obs = torch.from_numpy(new_obs)

In [ ]:
# environment_times_steps = test_data.shape[0]
# steps_multiplicator = 1
# timesteps = 10 + (environment_times_steps * num_cpu) * steps_multiplicator 
# model.learn(total_timesteps=timesteps) # The total number of samples (env steps) to train on

In [26]:
venv = DummyVecEnv([_make_env for _ in range(7)])
#venv = SubprocVecEnv([_make_env for _ in range(7)])

# sample_iter = 7
# from sklearn.model_selection import train_test_split
# vec_list = []
# for i in range(sample_iter):
#     S_train, _, A_train, _ = train_test_split(
#         S, A, test_size=0.33, random_state=np.random.randint(0, 100000))
#     def _make_env(s=S_train,a=A_train,R=None,max_steps=None):
        
#         """Helper function to create a single environment. Put any logic here, but make sure to return a RolloutInfoWrapper."""
#         _env = CustomEnv(s,a,R,max_steps)
#         #_env = TimeLimit(_env, max_episode_steps=max_steps)
#         _env = RolloutInfoWrapper(_env)
#         return _env
#     vec_list.append(_make_env)

# venv = DummyVecEnv(vec_list)

In [27]:
# SAC1024Policy NonTrainablePolicy RandomPolicy 這沒辦法用
ALG = MlpPolicy # MlpPolicy ,ActorCriticPolicy, FeedForward32Policy, SAC1024Policy, NonTrainablePolicy, RandomPolicy, ZeroPolicy

# PPO
# MlpPolicy,ActorCriticPolicy, FeedForward32Policy, SAC1024Policy, NonTrainablePolicy, RandomPolicy, ZeroPolicy

In [28]:
from imitation.algorithms.adversarial.airl import AIRL

SEED = np.random.randint(0, 1000000)
venv.seed(SEED)
learner = PPO(
    policy=ALG,
    env=venv,
    seed=SEED, #下SEED會影響到INDEX抽樣# np.random.randint(0, self.S.shape[0])
)


reward_net = BasicRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)
#venv = util.make_vec_env("Pendulum-v1", n_envs=3, rng=rng)
rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)




rng = np.random.default_rng(seed=SEED)
rollouts = rollout.rollout(
    policy=learner,
    venv=rvenv, # 這邊要改嗎?
    sample_until = rollout.make_sample_until(min_episodes=50), # min_timesteps=FIXED_LENGTH, min_timesteps 這功能意義在哪?
    rng=rng
    )
# transitions = rollout.flatten_trajectories(rollouts) # 意義


airl_trainer = AIRL(
    demonstrations=rollouts,
    demo_batch_size=2048,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=16,
    venv=rvenv,
    gen_algo=learner,
    reward_net=reward_net,
    #allow_variable_horizon=True ### 這啥? # 官網不建議使用，FALSE這是防呆功能，因為會影響到表現
)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")


In [29]:
new_obs.shape

torch.Size([1, 27])

In [30]:
learner.predict(new_obs)[0].shape

(1, 53)

In [79]:
# for trajectory in rollouts:
#     print(f"obs, Trajectory length: {len(trajectory.obs)}")
#     print(f"acts, Trajectory length: {len(trajectory.acts)}")

In [31]:
airl_trainer.train(200_000)  # Train for 2_000_000 steps to match expert.

round:   0%|          | 0/13 [00:00<?, ?it/s]

---------------------------------------
| raw/                        |       |
|    gen/time/fps             | 4466  |
|    gen/time/iterations      | 1     |
|    gen/time/time_elapsed    | 3     |
|    gen/time/total_timesteps | 14336 |
---------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 8.61e-30 |
|    disc/disc_loss                   | 37.7     |
|    disc/disc_proportion_expert_pred | 1        |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 2.05e+03 |
|    disc/n_generated                 | 2.05e+03 |
--------------------------------------------------
--------------------------------------------------
| raw/

round:   8%|▊         | 1/13 [00:13<02:36, 13.04s/it]

--------------------------------------------------
| raw/                              |            |
|    gen/time/fps                   | 4617       |
|    gen/time/iterations            | 1          |
|    gen/time/time_elapsed          | 3          |
|    gen/time/total_timesteps       | 28672      |
|    gen/train/approx_kl            | 0.01852577 |
|    gen/train/clip_fraction        | 0.245      |
|    gen/train/clip_range           | 0.2        |
|    gen/train/entropy_loss         | -75.2      |
|    gen/train/explained_variance   | -0.000366  |
|    gen/train/learning_rate        | 0.0003     |
|    gen/train/loss                 | 8.18e+04   |
|    gen/train/n_updates            | 10         |
|    gen/train/policy_gradient_loss | -0.0369    |
|    gen/train/std                  | 0.999      |
|    gen/train/value_loss           | 1.71e+05   |
--------------------------------------------------
--------------------------------------------------
| raw/                         

round:  15%|█▌        | 2/13 [00:26<02:23, 13.03s/it]

...

round:  85%|████████▍ | 11/13 [02:25<00:26, 13.41s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_rew_wrapped_mean | -7.2e+04   |
|    gen/time/fps                    | 4656       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 3          |
|    gen/time/total_timesteps        | 172032     |
|    gen/train/approx_kl             | 0.03835406 |
|    gen/train/clip_fraction         | 0.324      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -74.7      |
|    gen/train/explained_variance    | 6.5e-05    |
|    gen/train/learning_rate         | 0.0003     |
|    gen/train/loss                  | 3.6e+05    |
|    gen/train/n_updates             | 110        |
|    gen/train/policy_gradient_loss  | -0.0289    |
|    gen/train/std                   | 0.991      |
|    gen/train/value_loss            | 7.79e+05   |
---------------------------------------------------
------------

round:  92%|█████████▏| 12/13 [02:38<00:13, 13.22s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_rew_wrapped_mean | -7.2e+04    |
|    gen/time/fps                    | 4389        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 3           |
|    gen/time/total_timesteps        | 186368      |
|    gen/train/approx_kl             | 0.036408782 |
|    gen/train/clip_fraction         | 0.351       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -74.6       |
|    gen/train/explained_variance    | -4.36e-05   |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 3.61e+05    |
|    gen/train/n_updates             | 120         |
|    gen/train/policy_gradient_loss  | -0.0276     |
|    gen/train/std                   | 0.989       |
|    gen/train/value_loss            | 7.64e+05    |
----------------------------------------------

round: 100%|██████████| 13/13 [02:51<00:00, 13.17s/it]


In [32]:
#venv.seed(SEED)
# 這段不是 reward network的產出  venv
learner_rewards_after_training, _ = evaluate_policy(
    airl_trainer.policy, venv, 100, return_episode_rewards=True,
)
rlearner_rewards_after_training, _ = evaluate_policy(
    airl_trainer.policy, rvenv, 100, return_episode_rewards=True,
)
print("venv mean reward after training:", np.mean(learner_rewards_after_training))
print("rvenv mean reward after training:", np.mean(rlearner_rewards_after_training))

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


venv mean reward after training: 0.0
rvenv mean reward after training: -263710.32412467


In [ ]:
# airl_trainer.policy.predict(new_obs,deterministic=True)

In [33]:
from imitation.algorithms.adversarial.gail import GAIL

SEED = np.random.randint(0, 1000000)
venv.seed(SEED)
learner = PPO(
    policy=ALG,
    env=venv,
    seed=SEED, #下SEED會影響到INDEX抽樣# np.random.randint(0, self.S.shape[0])
)


reward_net = BasicRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)

rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)


rng = np.random.default_rng(seed=SEED)
rollouts = rollout.rollout(
    policy=learner,
    venv=rvenv, # 這邊要改嗎?
    sample_until = rollout.make_sample_until(min_episodes=50), # min_timesteps=FIXED_LENGTH, min_timesteps 這功能意義在哪?
    rng=rng
    )
# transitions = rollout.flatten_trajectories(rollouts) # 意義


gail_trainer = GAIL(
    demonstrations=rollouts, # transitions ?
    demo_batch_size=2048,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=16,
    venv=rvenv,
    gen_algo=learner,
    reward_net=reward_net,
    #allow_variable_horizon=True ####不知道這個要幹嘛，這千萬不能用，官網不建議使用
)

In [34]:
gail_trainer.train(200_000)  # Train for 800_000 steps to match expert.

round:   0%|          | 0/13 [00:00<?, ?it/s]

---------------------------------------
| raw/                        |       |
|    gen/time/fps             | 4223  |
|    gen/time/iterations      | 1     |
|    gen/time/time_elapsed    | 3     |
|    gen/time/total_timesteps | 14336 |
---------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.51     |
|    disc/disc_acc_expert             | 0.775    |
|    disc/disc_acc_gen                | 0.244    |
|    disc/disc_entropy                | 0.693    |
|    disc/disc_loss                   | 0.694    |
|    disc/disc_proportion_expert_pred | 0.766    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 2.05e+03 |
|    disc/n_generated                 | 2.05e+03 |
--------------------------------------------------
--------------------------------------------------
| raw/

round:   8%|▊         | 1/13 [00:13<02:42, 13.50s/it]

--------------------------------------------------
| raw/                              |            |
|    gen/time/fps                   | 4268       |
|    gen/time/iterations            | 1          |
|    gen/time/time_elapsed          | 3          |
|    gen/time/total_timesteps       | 28672      |
|    gen/train/approx_kl            | 0.03369863 |
|    gen/train/clip_fraction        | 0.354      |
|    gen/train/clip_range           | 0.2        |
|    gen/train/entropy_loss         | -75.2      |
|    gen/train/explained_variance   | -0.00456   |
|    gen/train/learning_rate        | 0.0003     |
|    gen/train/loss                 | 131        |
|    gen/train/n_updates            | 10         |
|    gen/train/policy_gradient_loss | -0.0347    |
|    gen/train/std                  | 1          |
|    gen/train/value_loss           | 954        |
--------------------------------------------------
--------------------------------------------------
| raw/                         

round:  15%|█▌        | 2/13 [00:26<02:26, 13.33s/it]

...

round:  85%|████████▍ | 11/13 [02:28<00:26, 13.48s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_rew_wrapped_mean | 4.17e+03   |
|    gen/time/fps                    | 3973       |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 3          |
|    gen/time/total_timesteps        | 172032     |
|    gen/train/approx_kl             | 0.16591904 |
|    gen/train/clip_fraction         | 0.661      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -75.1      |
|    gen/train/explained_variance    | -6.87      |
|    gen/train/learning_rate         | 0.0003     |
|    gen/train/loss                  | 0.0248     |
|    gen/train/n_updates             | 110        |
|    gen/train/policy_gradient_loss  | -0.0135    |
|    gen/train/std                   | 0.998      |
|    gen/train/value_loss            | 0.281      |
---------------------------------------------------
------------

round:  92%|█████████▏| 12/13 [02:41<00:13, 13.50s/it]

--------------------------------------------------
| raw/                               |           |
|    gen/rollout/ep_rew_wrapped_mean | 4.17e+03  |
|    gen/time/fps                    | 3780      |
|    gen/time/iterations             | 1         |
|    gen/time/time_elapsed           | 3         |
|    gen/time/total_timesteps        | 186368    |
|    gen/train/approx_kl             | 0.1497237 |
|    gen/train/clip_fraction         | 0.634     |
|    gen/train/clip_range            | 0.2       |
|    gen/train/entropy_loss          | -75.1     |
|    gen/train/explained_variance    | -1.23     |
|    gen/train/learning_rate         | 0.0003    |
|    gen/train/loss                  | 0.142     |
|    gen/train/n_updates             | 120       |
|    gen/train/policy_gradient_loss  | -0.0185   |
|    gen/train/std                   | 0.997     |
|    gen/train/value_loss            | 0.217     |
--------------------------------------------------
-------------------------------

round: 100%|██████████| 13/13 [02:55<00:00, 13.50s/it]


In [35]:
#venv.seed(SEED)
# 這段不是 reward network的產出  venv
learner_rewards_after_training, _ = evaluate_policy(
    gail_trainer.policy, venv, 100, return_episode_rewards=True,
)
rlearner_rewards_after_training, _ = evaluate_policy(
    gail_trainer.policy, rvenv, 100, return_episode_rewards=True,
)
print("venv mean reward after training:", np.mean(learner_rewards_after_training))
print("rvenv mean reward after training:", np.mean(rlearner_rewards_after_training))

venv mean reward after training: 0.0
rvenv mean reward after training: 472.88244828395545


In [36]:
from imitation.algorithms import preference_comparisons


SEED = np.random.randint(0,100000)

rng = np.random.default_rng(SEED)
venv.seed(SEED)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(
    warning_threshold=0,
    rng=rng
)
gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
preference_model = preference_comparisons.PreferenceModel(reward_net)
reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=preference_comparisons.CrossEntropyRewardLoss(),
    epochs=23,
    rng=rng,
)

rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

# Several hyperparameters (reward_epochs, ppo_clip_range, ppo_ent_coef,
# ppo_gae_lambda, ppo_n_epochs, discount_factor, use_sde, sde_sample_freq,
# ppo_lr, exploration_frac, num_iterations, initial_comparison_frac,
# initial_epoch_multiplier, query_schedule) used in this example have been
# approximately fine-tuned to reach a reasonable level of performance.
learner = PPO(
    policy=ALG,#FeedForward32Policy, # SAC1024Policy  MlpPolicy NonTrainablePolicy RandomPolicy ZeroPolicy
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=rvenv,
    seed=SEED,
    n_steps=2048 // rvenv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=learner,
    reward_fn=reward_net,
    venv=rvenv,
    exploration_frac=0.05,
    rng=rng,
)

pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=5,  # Set to 60 for better performance
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    fragment_length=100,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    initial_epoch_multiplier=4,
    query_schedule="hyperbolic",
)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2044`, after every 31 untruncated mini-batches, there will be a truncated mini-batch of size 60
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=292 and n_envs=7)
  warnings.warn(


In [37]:
pref_comparisons.train(
    total_timesteps=5_000,
    total_comparisons=200,
)

Query schedule: [20, 51, 41, 34, 29, 25]
Requested 3800 transitions but only 0 in buffer. Sampling 3800 additional transitions.
Sampling 200 exploratory transitions.
Creating fragment pairs
Gathering preferences
Dataset now contains 20 comparisons


Training reward model:   0%|          | 0/92 [00:00<?, ?it/s]

Training agent for 1000 timesteps
----------------------------------------------------
| raw/                                 |           |
|    agent/rollout/ep_rew_wrapped_mean | -4.85e+07 |
|    agent/time/fps                    | 4330      |
|    agent/time/iterations             | 1         |
|    agent/time/time_elapsed           | 0         |
|    agent/time/total_timesteps        | 2044      |
----------------------------------------------------
-------------------------------------------------------
| mean/                                   |           |
|    agent/rollout/ep_rew_wrapped_mean    | -4.85e+07 |
|    agent/time/fps                       | 4.33e+03  |
|    agent/time/iterations                | 1         |
|    agent/time/time_elapsed              | 0         |
|    agent/time/total_timesteps           | 2.04e+03  |
|    agent/train/approx_kl                | 0.292     |
|    agent/train/clip_fraction            | 0.867     |
|    agent/train/clip_range           

Training reward model:   0%|          | 0/23 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-----------------------------------------------------
| raw/                                 |            |
|    agent/rollout/ep_rew_wrapped_mean | -2.42e+07  |
|    agent/time/fps                    | 4220       |
|    agent/time/iterations             | 1          |
|    agent/time/time_elapsed           | 0          |
|    agent/time/total_timesteps        | 4088       |
|    agent/train/approx_kl             | 0.29181227 |
|    agent/train/clip_fraction         | 0.867      |
|    agent/train/clip_range            | 0.1        |
|    agent/train/entropy_loss          | -75.9      |
|    agent/train/explained_variance    | -0.337     |
|    agent/train/learning_rate         | 0.002      |
|    agent/train/loss                  | -0.573     |
|    agent/train/n_updates             | 10         |
|    agent/train/policy_gradient_loss  | -0.0035    |
|    agent/train/std                   | 1.02       |
|    agent/train/value_loss            | 1.09   

...

Training reward model:   0%|          | 0/23 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-----------------------------------------------------
| raw/                                 |            |
|    agent/rollout/ep_rew_wrapped_mean | -9.7e+06   |
|    agent/time/fps                    | 4293       |
|    agent/time/iterations             | 1          |
|    agent/time/time_elapsed           | 0          |
|    agent/time/total_timesteps        | 10220      |
|    agent/train/approx_kl             | 0.42662865 |
|    agent/train/clip_fraction         | 0.824      |
|    agent/train/clip_range            | 0.1        |
|    agent/train/entropy_loss          | -78.5      |
|    agent/train/explained_variance    | 0.936      |
|    agent/train/learning_rate         | 0.002      |
|    agent/train/loss                  | -0.764     |
|    agent/train/n_updates             | 40         |
|    agent/train/policy_gradient_loss  | 0.0141     |
|    agent/train/std                   | 1.08       |
|    agent/train/value_loss            | 1.11   

Training reward model:   0%|          | 0/23 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-----------------------------------------------------
| raw/                                 |            |
|    agent/rollout/ep_rew_wrapped_mean | -8.08e+06  |
|    agent/time/fps                    | 4302       |
|    agent/time/iterations             | 1          |
|    agent/time/time_elapsed           | 0          |
|    agent/time/total_timesteps        | 12264      |
|    agent/train/approx_kl             | 0.25147724 |
|    agent/train/clip_fraction         | 0.812      |
|    agent/train/clip_range            | 0.1        |
|    agent/train/entropy_loss          | -79.6      |
|    agent/train/explained_variance    | 0.954      |
|    agent/train/learning_rate         | 0.002      |
|    agent/train/loss                  | -0.817     |
|    agent/train/n_updates             | 50         |
|    agent/train/policy_gradient_loss  | -0.00847   |
|    agent/train/std                   | 1.09       |
|    agent/train/value_loss            | 1.9    

{'reward_loss': 7.051157639840053e-05, 'reward_accuracy': 1.0}

In [38]:
reward, _ = evaluate_policy(learner.policy,rvenv, 100)
print("Reward:", reward)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Reward: -6205.705026188307


In [33]:
venv

In [39]:
from imitation.algorithms import bc
SEED = np.random.randint(0,1000000)
rng = np.random.default_rng(SEED)
venv.seed(SEED)




reward_net = BasicRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,

)
rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)
learner = PPO(
    policy=ALG,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=SEED,
    n_steps=1024, #2048 // rvenv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)




rollouts = rollout.rollout(
    learner,
    venv,
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)
#transitions = rollout.flatten_trajectories(rollouts)

bc_trainer = bc.BC(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    demonstrations=rollouts, # 可放這個，但意義???? transitions 
    rng=rng,
)
bc_trainer.train(n_epochs=10)




0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.0752  |
|    entropy        | 75.2     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 119      |
|    loss           | 73.3     |
|    neglogp        | 73.3     |
|    prob_true_act  | 6.26e-31 |
|    samples_so_far | 32       |
--------------------------------


498batch [00:01, 299.74batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.0752  |
|    entropy        | 75.2     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 117      |
|    loss           | 73.9     |
|    neglogp        | 74       |
|    prob_true_act  | 7.61e-31 |
|    samples_so_far | 16032    |
--------------------------------


982batch [00:03, 295.88batch/s]

...

86497batch [05:00, 297.97batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 86500    |
|    ent_loss       | -0.0752  |
|    entropy        | 75.2     |
|    epoch          | 9        |
|    l2_loss        | 0        |
|    l2_norm        | 117      |
|    loss           | 75.8     |
|    neglogp        | 75.8     |
|    prob_true_act  | 1.8e-30  |
|    samples_so_far | 2768032  |
--------------------------------


86984batch [05:02, 294.44batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 87000    |
|    ent_loss       | -0.0753  |
|    entropy        | 75.3     |
|    epoch          | 9        |
|    l2_loss        | 0        |
|    l2_norm        | 117      |
|    loss           | 74.2     |
|    neglogp        | 74.3     |
|    prob_true_act  | 1.45e-29 |
|    samples_so_far | 2784032  |
--------------------------------


87480batch [05:03, 287.93batch/s]


In [41]:
rreward, _ = evaluate_policy(bc_trainer.policy,rvenv, 100, return_episode_rewards=True)
reward, _ = evaluate_policy(bc_trainer.policy,venv, 100, return_episode_rewards=True)
print("rvenv Reward:", np.mean(rreward))
print("venv Reward:", np.mean(reward))

rvenv Reward: -29540.611963680385
venv Reward: 0.0


In [46]:
from imitation.algorithms.dagger import SimpleDAggerTrainer, DAggerTrainer

SEED = np.random.randint(0, 1000000)
rng = np.random.default_rng(SEED)

venv.seed(SEED)
reward_net = BasicRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)
#venv = util.make_vec_env("Pendulum-v1", n_envs=3, rng=rng)
rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)


learner = PPO(
    policy=ALG,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=rvenv,
    seed=SEED,
    n_steps=2048 // rvenv.num_envs,
    batch_size=64, # n_steps * n_envs
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)


bc_trainer = bc.BC(
    observation_space=rvenv.observation_space,
    action_space=venv.action_space,
    rng=rng,
)
#with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    #print(tmpdir)
simple_dagger_trainer = SimpleDAggerTrainer(
    venv=rvenv,
    scratch_dir=0,#tmpdir,
    expert_policy=learner,
    bc_trainer=bc_trainer,
    rng=rng,
)


dagger_trainer = DAggerTrainer(
    venv=rvenv,
    scratch_dir=0,#tmpdir,
    #expert_policy=learner,
    bc_trainer=bc_trainer,
    rng=rng,
)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\ppo\ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2044`, after every 31 untruncated mini-batches, there will be a truncated mini-batch of size 60
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=292 and n_envs=7)
  warnings.warn(


In [47]:
simple_dagger_trainer.train(100)
reward, _ = evaluate_policy(simple_dagger_trainer.policy, rvenv, 10)
print("Reward:", reward)

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 9 and the array at index 2 has size 27

In [16]:
dagger_trainer.policy

FeedForward32Policy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=9, out_features=32, bias=True)
      (1): Tanh()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=9, out_features=32, bias=True)
      (1): Tanh()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=32, out_features=23, bias=True)
  (value_net): Linear(in_features=32, out_features=1, bias=True)
)

In [44]:
from imitation.algorithms import density 

SEED = np.random.randint(0, 1000000)
rng = np.random.default_rng(SEED)
venv.seed(SEED)



reward_net = BasicRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)
#venv = util.make_vec_env("Pendulum-v1", n_envs=3, rng=rng)
rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)



learner = PPO(
    policy=ALG,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=rvenv,
    seed=SEED,
    n_steps=2048 // rvenv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)

rollouts = rollout.rollout(
    policy=learner,
    venv=venv, # vrvenv這邊要改嗎?
    sample_until = rollout.make_sample_until(min_episodes=50), # min_timesteps=FIXED_LENGTH, min_timesteps 這功能意義在哪?
    rng=rng
    )

density_trainer = density.DensityAlgorithm(
    venv=venv,
    rng=rng,
    demonstrations=rollouts,
    rl_algo=learner,
    density_type=density.DensityType.STATE_ACTION_DENSITY,
    is_stationary=True,
    kernel="gaussian",
    kernel_bandwidth=0.4,
    standardise_inputs=True,
)

density_trainer.train()


In [18]:
density_trainer.train_policy(100)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitat

In [45]:
#evaluate_policy(density_trainer.policy)
reward, _ = evaluate_policy(density_trainer.policy, rvenv, 100)
print("Reward:", reward)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\imitation\data\types.py:279: UserWarning: tried to wrap <class 'torch.Tensor'> as an observation
  warnings.warn(f"tried to wrap {type(obs)} as an observation")


Reward: -19146.625607714057


In [ ]:
# from imitation.algorithms import sqil
# #from imitation.data import huggingface_utils

# # Download some expert trajectories from the HuggingFace Datasets Hub.
# #dataset = datasets.load_dataset("HumanCompatibleAI/ppo-CartPole-v1")
# #rollouts = huggingface_utils.TrajectoryDatasetSequence(dataset["train"])


# venv = DummyVecEnv([_make_env for _ in range(20)])


# SEED = np.random.randint(0, 1000000)
# rng = np.random.default_rng(SEED)

# venv.seed(SEED)

# # reward_net = BasicRewardNet(
# #     observation_space=venv.observation_space,
# #     action_space=venv.action_space,
# #     normalize_input_layer=RunningNorm,
# # )
# #venv = util.make_vec_env("Pendulum-v1", n_envs=3, rng=rng)
# #rvenv = RewardVecEnvWrapper(venv, reward_net.predict_processed)


# ##################
# # SQIL
# # 這個只能用 gymnasium.spaces.discrete.Discrete'
# #
# #
# #################
# learner = PPO(
#     ActorCriticPolicy, rvenv, learning_rate=3e-4, gamma=0.95, ent_coef=1e-4, n_steps=2048,seed=SEED
# )

# rollouts = rollout.rollout(
#     policy=learner,
#     venv=venv, # 這邊要改嗎?
#     sample_until = rollout.make_sample_until(min_episodes=50), # min_timesteps=FIXED_LENGTH, min_timesteps 這功能意義在哪?
#     rng=rng
#     )



# sqil_trainer = sqil.SQIL(
#     policy=learner,
#     venv=venv,
#     demonstrations=rollouts,
# )
# # Hint: set to 1_000_000 to match the expert performance.
# sqil_trainer.train(total_timesteps=1_000)